In [102]:
import pandas as pd
import numpy as np
from random import shuffle

In [3]:
data = pd.read_csv('sampleData.csv', header=0)

In [4]:
data

,Name,Kika,Thelonius,SmokeShip,Ole,MeadHall,IndiaPavilion,VeggieGalaxy,Area4,PaintedBurro,RusselHouse,Christophers,RylesJazz,CambridgeCommons
0,A,2.0,3.0,NaN,NaN,NaN,1.0,NaN,1.0,2.0,3.0,NaN,NaN,NaN
1,B,2.0,NaN,1.0,NaN,3.0,NaN,NaN,NaN,3.0,NaN,2.0,1.0,NaN
2,C,1.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,1.0
3,D,1.0,3.0,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,3.0
4,E,3.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,F,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,G,NaN,NaN,NaN,NaN,3.0,1.0,2.0,NaN,NaN,3.0,NaN,1.0,2.0
7,H,2.0,1.0,3.0,NaN,NaN,NaN,NaN,3.0,1.0,2.0,NaN,NaN,NaN
8,I,NaN,NaN,NaN,3.0,1.0,2.0,NaN,1.0,NaN,NaN,3.0,NaN,2.0
9,J,NaN,NaN,1.0,2.0,3.0,NaN,NaN,NaN,NaN,1.0,2.0,3.0,NaN


In [5]:
dinner = data.loc[:,'Name':'VeggieGalaxy']

In [6]:
brunch = data.loc[:,'Area4':]
names = data['Name']
brunch = pd.concat([names,brunch], axis=1)

In [122]:
cap = 3
kika = []
thelonious = []
smokeshop =[]
ole = []
meadhall = []
indiapav = []
veggieGalaxy = []
names = data['Name'].tolist()
assigned = []
unassigned = []

assignmentLists = [kika,thelonious,smokeshop,ole,meadhall,indiapav,veggieGalaxy]
restaurantNames = dinner.columns[1:].tolist()
assignmentDict = dict(zip(restaurantNames, assignmentLists))

zeroes = [0]*len(names)

preferenceScoreDict = dict(zip(names, zeroes))

In [123]:
for col in dinner.columns[1:]:
    d = dinner[col]
    i = 0
    while i < len(d):
        if d[i] == 1:
            #assign first preference
            assignmentDict[col].append(names[i])
            #mark the person as assigned. 
            assigned.append(names[i])
            #update the person's cumulative preference score
            preferenceScoreDict[names[i]] += 1
        i += 1
#Now all first choices have been granted, check whether any restaurants are over capacity

def dinnerCapacityCheck():
    dinnerOverassigned = []
    dinnerUnderassigned = []
    dinnerFull = []
    for rest in restaurantNames:
        if len(assignmentDict[rest]) > cap:
            print(str(rest) + ' is overassigned')
            dinnerOverassigned.append(rest)
        elif len(assignmentDict[rest]) < cap:
            print(str(rest) + ' is underassigned')
            dinnerUnderassigned.append(rest)
        elif len(assignmentDict[rest]) == cap:
            print(str(rest) + ' is full')
            dinnerFull.append(rest)
    return dinnerOverassigned, dinnerUnderassigned, dinnerFull
dinnerOverassigned, dinnerUnderassigned, dinnerFull = dinnerCapacityCheck()
dinnerDone=False
if len(dinnerOverassigned) == 0:
    #everyone gets their first choice, ideal solution.
    dinnerDone = True
    for name in restaurantNames:
        print("The following have been assigned to " + str(name) + ":")
        for x in assignmentDict[name]:
            print(str(x) + '\n')
elif len(dinnerOverassigned) > 0:
    #print(dinnerOverassigned)
    #print(assignmentDict)
    #some people will have to be reassigned. 
    #for each overassigned restaurant, randomly choose the number of people required to kick out to go below the cap
    for rest in dinnerOverassigned:
        dinnerList = assignmentDict[rest]
        bumpNum = len(dinnerList)-cap
        gotBumped = np.random.choice(dinnerList, bumpNum)
        for b in gotBumped:
            assignmentDict[rest].remove(b) #kick that dude out
            assigned.remove(b) #mark them unassigned 
            unassigned.append(b)
            preferenceScoreDict[b] += 1 #increment their preference score
        #print(assignmentDict)
        #assign bumped people to their second choice.
        #print(unassigned)
        for x in unassigned:
            if x in assigned:
                print(str(x) + " is already assigned.")
                unassigned.remove(x)
                continue
            #print(x)
            i = names.index(x)
            row = dinner.ix[i] #this should be the row of the the bumped person
            #reassign them to their second choice
            j=0
            #find the row index of this person who must be reassigned. 
            while j < len(dinner['Name']):
                if dinner['Name'][j] == x:
                    break
                j += 1
            #Search for which restaurant was their second choice. 
            for col in dinner.columns[1:]:
                d = dinner[col][j]
                if d == 2:
                    #assign second preference
                    assignmentDict[col].append(names[j])
                    #mark the person as assigned. 
                    if names[j] not in assigned:
                        assigned.append(names[j])
        for a in assigned:
            if a in unassigned:
                unassigned.remove(a)

    dinnerOverassigned, dinnerUnderassigned, dinnerFull = dinnerCapacityCheck()


Kika is full
Thelonius is underassigned
SmokeShip is full
Ole is underassigned
MeadHall is underassigned
IndiaPavilion is overassigned
VeggieGalaxy is underassigned
Kika is full
Thelonius is underassigned
SmokeShip is full
Ole is underassigned
MeadHall is underassigned
IndiaPavilion is full
VeggieGalaxy is underassigned


In [124]:
print(preferenceScoreDict)

{'F': 0, 'M': 1, 'B': 1, 'G': 2, 'I': 1, 'P': 1, 'D': 1, 'A': 1, 'E': 1, 'H': 1, 'L': 1, 'O': 1, 'K': 1, 'N': 1, 'J': 1, 'C': 1}


In [125]:
#now second choices have been assigned. 
if len(dinnerOverassigned) == 0 and dinnerDone==False:
    dinnerDone = True
    for name in restaurantNames:
        print("The following have been assigned to " + str(name) + ":")
        for x in assignmentDict[name]:
            print(str(x) + '\n')

#consider bumping people to third choice.
elif len(dinnerOverassigned) > 0 and dinnerDone==False:
    #some people will have to be reassigned. 
    #for each overassigned restaurant, randomly choose the number of people required to kick out to go below the cap
    for rest in dinnerOverassigned:
        dinnerList = assignmentDict[rest]
        bumpNum = len(dinnerList)-cap
        bumpOrder = random.shuffle(dinnerList) #want to preferentially bump people who haven't yet been bumped.
        gotBumped = []
        for x in bumpOrder:
            if len(gotBumped) == bumpNum:
                break
            if preferenceScoreDict[x] == 1:
                gotBumped.append(x)
            else:
                continue
        for b in gotBumped:
            assignmentDict[rest].remove(b) #kick that dude out
            assigned.remove(b) #mark them unassigned 
            unassigned.append(b)
            preferenceScoreDict[b] += 1 #increment their preference score
        #print(assignmentDict)
        #assign bumped people to their second choice.
        #print(unassigned)
        for x in unassigned:
            if x in assigned:
                print(str(x) + " is already assigned.")
                unassigned.remove(x)
                continue
            #print(x)
            i = names.index(x)
            row = dinner.ix[i] #this should be the row of the the bumped person
            #reassign them to their second choice
            j=0
            #find the row index of this person who must be reassigned. 
            while j < len(dinner['Name']):
                if dinner['Name'][j] == x:
                    break
                j += 1
            #Search for which restaurant was their third choice. 
            for col in dinner.columns[1:]:
                d = dinner[col][j]
                if d == 3:
                    #assign second preference
                    assignmentDict[col].append(names[j])
                    #mark the person as assigned. 
                    if names[j] not in assigned:
                        assigned.append(names[j])
        for a in assigned:
            if a in unassigned:
                unassigned.remove(a)

    dinnerOverassigned, dinnerUnderassigned, dinnerFull = dinnerCapacityCheck()

    #if still overassigned after the second choice, run the brunch algorithm
    #then minimize preference scores as a last resort. 
            
if len(dinnerOverassigned) == 0 and dinnerDone==False:
    dinnerDone = True
    for name in restaurantNames:
        print("The following have been assigned to " + str(name) + ":")
        for x in assignmentDict[name]:
            print(str(x) + '\n')  


The following have been assigned to Kika:
C

D

N

The following have been assigned to Thelonius:
H

O

The following have been assigned to SmokeShip:
B

J

P

The following have been assigned to Ole:
E

The following have been assigned to MeadHall:
I

The following have been assigned to IndiaPavilion:
A

L

M

The following have been assigned to VeggieGalaxy:
K

G



In [126]:
brunch.columns[1:].tolist()

['Area4',
 'PaintedBurro',
 'RusselHouse',
 'Christophers',
 'RylesJazz',
 'CambridgeCommons']

In [127]:
Area4 = []
PaintedBurro = []
RusselHouse =[]
Christophers = []
Ryles = []
CambridgeCommons = []

assigned = []
unassigned = []

assignmentLists = [Area4,PaintedBurro,RusselHouse,Christophers,Ryles,CambridgeCommons]
restaurantNames = brunch.columns[1:].tolist()
assignmentDict = dict(zip(restaurantNames, assignmentLists))

#zeroes = [0]*len(names)
#preferenceScoreDict = dict(zip(names, zeroes))

In [128]:
for col in brunch.columns[1:]:
    d = brunch[col]
    i = 0
    while i < len(d):
        if d[i] == 1:
            #assign first preference
            assignmentDict[col].append(names[i])
            #mark the person as assigned. 
            assigned.append(names[i])
            #update the person's cumulative preference score
            preferenceScoreDict[names[i]] += 1
        i += 1
#Now all first choices have been granted, check whether any restaurants are over capacity

def brunchCapacityCheck():
    brunchOverassigned = []
    brunchUnderassigned = []
    brunchFull = []
    for rest in restaurantNames:
        if len(assignmentDict[rest]) > cap:
            print(str(rest) + ' is overassigned')
            brunchOverassigned.append(rest)
        elif len(assignmentDict[rest]) < cap:
            print(str(rest) + ' is underassigned')
            brunchUnderassigned.append(rest)
        elif len(assignmentDict[rest]) == cap:
            print(str(rest) + ' is full')
            brunchFull.append(rest)
    return brunchOverassigned, brunchUnderassigned, brunchFull
brunchOverassigned, brunchUnderassigned, brunchFull = brunchCapacityCheck()
brunchDone=False
if len(brunchOverassigned) == 0:
    #everyone gets their first choice, ideal solution.
    brunchDone = True
    for name in restaurantNames:
        print("The following have been assigned to " + str(name) + ":")
        for x in assignmentDict[name]:
            print(str(x) + '\n')
elif len(brunchOverassigned) > 0:
    #print(dinnerOverassigned)
    #print(assignmentDict)
    #some people will have to be reassigned. 
    #for each overassigned restaurant, randomly choose the number of people required to kick out to go below the cap
    for rest in brunchOverassigned:
        brunchList = assignmentDict[rest]
        bumpNum = len(brunchList)-cap
        gotBumped = np.random.choice(brunchList, bumpNum)
        for b in gotBumped:
            assignmentDict[rest].remove(b) #kick that dude out
            assigned.remove(b) #mark them unassigned 
            unassigned.append(b)
            preferenceScoreDict[b] += 1 #increment their preference score
        #print(assignmentDict)
        #assign bumped people to their second choice.
        #print(unassigned)
        for x in unassigned:
            if x in assigned:
                print(str(x) + " is already assigned.")
                unassigned.remove(x)
                continue
            #print(x)
            i = names.index(x)
            row = brunch.ix[i] #this should be the row of the the bumped person
            #reassign them to their second choice
            j=0
            #find the row index of this person who must be reassigned. 
            while j < len(brunch['Name']):
                if brunch['Name'][j] == x:
                    break
                j += 1
            #Search for which restaurant was their second choice. 
            for col in brunch.columns[1:]:
                d = brunch[col][j]
                if d == 2:
                    #assign second preference
                    assignmentDict[col].append(names[j])
                    #mark the person as assigned. 
                    if names[j] not in assigned:
                        assigned.append(names[j])
        for a in assigned:
            if a in unassigned:
                unassigned.remove(a)

    brunchOverassigned, brunchUnderassigned,brunchFull = brunchCapacityCheck()

if len(brunchOverassigned) == 0 and brunchDone==False:
    brunchDone = True
    for name in restaurantNames:
        print("The following have been assigned to " + str(name) + ":")
        for x in assignmentDict[name]:
            print(str(x) + '\n')

#consider bumping people to third choice.
elif len(brunchOverassigned) > 0 and brunchDone==False:
    #some people will have to be reassigned. 
    #for each overassigned restaurant, randomly choose the number of people required to kick out to go below the cap
    for rest in brunchOverassigned:
        brunchList = assignmentDict[rest]
        bumpNum = len(brunchList)-cap
        bumpOrder = random.shuffle(brunchList) #want to preferentially bump people who haven't yet been bumped.
        gotBumped = []
        for x in bumpOrder:
            if len(gotBumped) == bumpNum:
                break
            if preferenceScoreDict[x] == 1:
                gotBumped.append(x)
            else:
                continue
        if len(gotBumped) < bumpNum:
            for x in bumpOrder:
                if len(gotBumped) == bumpNum:
                    break
                if preferenceScoreDict[x] == 2:
                    gotBumped.append(x)
                else:
                    continue
        if len(gotBumped) < numpNum:
            for x in bumpOrder:
                if len(gotBumped) == bumpNum:
                    break
                if preferenceScoreDict[x] == 3:
                    gotBumped.append(x)
                else:
                    continue
        for b in gotBumped:
            assignmentDict[rest].remove(b) #kick that dude out
            assigned.remove(b) #mark them unassigned 
            unassigned.append(b)
            preferenceScoreDict[b] += 1 #increment their preference score
        #print(assignmentDict)
        #assign bumped people to their second choice.
        #print(unassigned)
        for x in unassigned:
            if x in assigned:
                print(str(x) + " is already assigned.")
                unassigned.remove(x)
                continue
            #print(x)
            i = names.index(x)
            row = brunch.ix[i] #this should be the row of the the bumped person
            #reassign them to their second choice
            j=0
            #find the row index of this person who must be reassigned. 
            while j < len(brunch['Name']):
                if brunch['Name'][j] == x:
                    break
                j += 1
            #Search for which restaurant was their third choice. 
            for col in brunch.columns[1:]:
                d = brunch[col][j]
                if d == 3:
                    #assign second preference
                    assignmentDict[col].append(names[j])
                    #mark the person as assigned. 
                    if names[j] not in assigned:
                        assigned.append(names[j])
        for a in assigned:
            if a in unassigned:
                unassigned.remove(a)

    brunchOverassigned, brunchUnderassigned, brunchFull = brunchCapacityCheck()

if len(brunchOverassigned) == 0 and brunchDone==False:
    brunchDone = True
    for name in restaurantNames:
        print("The following have been assigned to " + str(name) + ":")
        for x in assignmentDict[name]:
            print(str(x) + '\n')

if dinnerDone == True and brunchDone == True:
    print("all these motherfuckers gonna eat")
else:
    print("impossible to fairly assign based on chosen preferences.")

Area4 is underassigned
PaintedBurro is full
RusselHouse is underassigned
Christophers is underassigned
RylesJazz is full
CambridgeCommons is full
The following have been assigned to Area4:
A

I

The following have been assigned to PaintedBurro:
H

K

N

The following have been assigned to RusselHouse:
J

The following have been assigned to Christophers:
D

L

The following have been assigned to RylesJazz:
B

G

O

The following have been assigned to CambridgeCommons:
C

M

P

all these motherfuckers gonna eat
